In [ ]:
import pandas as pd
import numpy as np
import json
from glob import glob
import os

from datasets import Dataset, Audio, load_dataset
from tqdm import tqdm

In [ ]:
DATASET_NAME = "mozilla-foundation/common_voice_11_0"
NAME = "cs"
SPLIT = "test"
SAMPLING_RATE = 16_000

In [ ]:
hf_dataset = load_dataset(DATASET_NAME, NAME, split=SPLIT)
hf_dataset = hf_dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))

In [ ]:
wav_lenghts = np.array([len(example['audio']['array']) for example in tqdm(hf_dataset.to_iterable_dataset(), total=len(hf_dataset))])

In [ ]:
wav_lenghts_secs = wav_lenghts/SAMPLING_RATE

In [ ]:
np.mean(wav_lenghts_secs)

In [ ]:
total_secs = np.sum(wav_lenghts_secs)
total_secs, total_secs/3600

Training table sumarisation

In [ ]:
model = "t5"
train_paths = glob(f'/home/sulcm/models/{model}/*')

In [ ]:
def format_latex_table(results: dict, best_results: dict) -> str:
    formated_table = ''
    for run, metrics in results.items():
        formated_table += run + ' & ' + ' & '.join([f'\\textbf{{{v}}}' if run == best_results[m][0] else v for m, v in metrics.items()]) + ' \\\\\n'
    
    return formated_table

In [ ]:
metrics = ['wer', 'cer']
best_results = dict.fromkeys(metrics, ('', 1.0))
table_prep = {}

for train_res in train_paths:
    path2res = train_res + '/all_results.json'
    if not os.path.exists(path2res):
        continue

    with open(path2res, 'r') as f:
        results = json.load(f)
    
    table_prep[train_res.split('/')[-1].split('-')[-1]] = {m: f'{100.0*results[f"eval_{m}"]:.02f}' for m in metrics}

    for metric in metrics:
        if best_results[metric][1] > results[f"eval_{metric}"]:
            best_results[metric] = (train_res.split('/')[-1].split('-')[-1], results[f"eval_{metric}"])

print(table_prep)
print(best_results)

In [ ]:
print(format_latex_table(table_prep, best_results))